In [3]:
import pandas as pd
from numpy import arange
import codecs

In [4]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import pickle

In [ ]:
commentairesName="/Users/gilles/Copy/Recherche/Rugby/Videos/CorrespondancesAction-PlayVsColour.csv"
commentairesTypesName="/Users/gilles/Copy/Recherche/Rugby/CorpusRugby2007/Timeline-TypeCommentaires.pkl"
commentaireActionName="/Users/gilles/Copy/Recherche/Rugby/CorpusRugby2007/Timeline-CommentaireAction.pkl"
commentairesFile=codecs.open(commentairesName,"r",encoding="utf8")
commentaires=commentairesFile.readlines()

In [ ]:
initialTime="00:11:52.4"
decalage=0
def decodeTime(temps):
    (heure,minute,seconde)=temps.split(":")
    result=int(100*(3600*int(heure)+60*int(minute)+float(seconde))-decalage)
    return result

decalage=decodeTime(initialTime)
print decalage, decodeTime(initialTime)

In [ ]:
def sliceDataFrame(df,debut,fin,seuil,colonnes):
    return df[df.index<fin][df[df.index<fin].index>=debut][colonnes].dropna(thresh=seuil)

In [ ]:
#timeLine=pd.DataFrame(index=[float(i)/100 for i in range(6500000)],columns=["début","fin","V1","V2","V3"])
timeLine=pd.DataFrame(columns=["début","fin","numEvent","événement","numAct","action","numSpk","Spk","texte","PxP","colour"])

In [ ]:
for ligne in commentaires[1:]:
    elements=ligne.strip().split(";")

    if elements[14]=="":
        typeCommentaire=elements[13]
    else:
        typeCommentaire=elements[14]

    if typeCommentaire in ["Play by play","Ponctuant"]:
        boolPlay=1
        boolColour=np.nan
    elif typeCommentaire in ["Colour commentary","Color commentary","Color Commentary"]:
        boolPlay=np.nan
        boolColour=1
    elif typeCommentaire=="":
        boolPlay=np.nan
        boolColour=np.nan
    else:
        print "commentaire non-standard",typeCommentaire
    
    if " " in elements[5]:
        (numEvent,Event)=elements[5].split(" ",1)
        numEvent=int(numEvent)
    else:
        (numEvent,Event)=(np.nan,np.nan)
    if " " in elements[6]:
        (numAction,Action)=elements[6].split(" ",1)
        numAction=int(numAction.strip("A-"))
    else:
        (numAction,Action)=(np.nan,np.nan)
    if " " in elements[8]:
        (numSpk,texte)=elements[8].split(" ",1)
        if "-" in numSpk:
            (spk,numSpk)=numSpk.split("-",1)
            numSpk=int(numSpk)
        else:
            numSpk=np.nan
            spk=numSpk
    else:
        (numSpk,texte)=(np.nan,np.nan)

    timeLine.loc[decodeTime(elements[1])]=[decodeTime(elements[1]),decodeTime(elements[2]),numEvent,Event,numAction,Action, numSpk,spk,texte, boolPlay, boolColour]

In [ ]:
timeLine[timeLine.index<3400].dropna(subset=["PxP"]).iloc[-1]["fin"]

In [ ]:
timeLine

In [ ]:
timeLine[timeLine["numAction"].notnull()]["textAction"]=timeLine["action"].str.split(" ").str[1:]
timeLine[timeLine["numAction"].notnull()]

In [ ]:
timeLine

In [ ]:
#sliceDataFrame(timeLine,3200,3300,1,["colour"])

In [ ]:
%%time
timeCommentType=pd.DataFrame(columns=["PxP","colour","sumPxP","sumColour"])
step=100
for i in arange(3000,644000,step):
    if len(sliceDataFrame(timeLine,i,i+step,1,["PxP"])) > 0:
#        print "PxP",i
        PxP=1
        sumPxP=sliceDataFrame(timeLine,i,i+step,1,["PxP"])["PxP"].sum()
    else:
        PxP=0
        sumPxP=0
    if len(timeLine[timeLine.index<i].dropna(subset=["PxP"]))>0:
        if timeLine[timeLine.index<i].dropna(subset=["PxP"]).iloc[-1]["fin"]>i:
            PxP=1
            sumPxP+=1
#            print "PxP+",i,PxP
    if len(sliceDataFrame(timeLine,i,i+step,1,["colour"])) > 0:
#        print "colour",i
        colour=1
        sumColour=sliceDataFrame(timeLine,i,i+step,1,["colour"])["colour"].sum()
    else:
        colour=0
        sumColour=0
    if len(timeLine[timeLine.index<i].dropna(subset=["colour"]))>0:
        if timeLine[timeLine.index<i].dropna(subset=["colour"]).iloc[-1]["fin"]>i:
            colour=1
            sumColour+=1
    timeCommentType.loc[i]=[PxP, colour, sumPxP, sumColour]

In [ ]:
#timeCommentType[0:5880][timeCommentType["sumPxP"]>1]
timeCommentType

In [ ]:
with codecs.open(commentairesTypesName,"wb") as output:
    pickle.dump(timeCommentType, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
with codecs.open(commentaireActionName,"wb") as output:
    pickle.dump(timeLine[["début","fin","numEvent","événement","numAct","action","numSpk","Spk","texte"]], output, pickle.HIGHEST_PROTOCOL)

In [ ]:
timeLine